# **Tree crown detection**

## pip libraries

In [ ]:
#core libraries
#!pip -q install scikit-image
#!pip -q install opencv-python
#!pip -q install pycurl
!pip -q install git+https://github.com/ESM-VFC/intake_zenodo_fetcher.git ##Intake Zenodo Fetcher


In [ ]:
!pip -q install DeepForest

## load libraries

In [ ]:
#load libraries
import glob
import os
import urllib

import intake
from intake_zenodo_fetcher import download_zenodo_files_for_entry
import matplotlib.pyplot as plt
import xmltodict
import cv2

%matplotlib inline

## create and load intake catalogue for images

In [ ]:
%%writefile tc.yaml
sources:
  NEONTREE_rgb:
    driver: xarray_image
    description: 'NeonTreeEvaluation RGB images (collection)'
    metadata:
      zenodo_doi: "10.5281/zenodo.3459803"
    args:
      urlpath: "{{ CATALOG_DIR }}/NEONsample_RGB/2018_MLBS_3_541000_4140000_image_crop.tif"

In [ ]:
cat_tc = intake.open_catalog('tc.yaml')

In [ ]:
for catalog_entry in list(cat_tc):
    download_zenodo_files_for_entry(
        cat_tc[catalog_entry],
        force_download=False
    )

In [ ]:
tc_rgb = cat_tc["NEONTREE_rgb"].to_dask()

In [ ]:
tc_rgb

## load and prepare labels

In [ ]:
filenames = glob.glob('./NEONsample_RGB/*.tif')
filesn = [os.path.basename(i) for i in filenames]

In [ ]:
##Create ordered dictionary of .xml annotation files
def loadxml(imagename):
  imagename = imagename.replace('.tif','')
  fullurl = "https://raw.githubusercontent.com/weecology/NeonTreeEvaluation/master/annotations/" + imagename + ".xml"
  file = urllib.request.urlopen(fullurl)
  data = file.read()
  file.close()
  data = xmltodict.parse(data)
  return data

allxml = [loadxml(i) for i in filesn]


In [ ]:
# function to extract bounding boxes
def extractbb(i):
  bb = [f['bndbox'] for f in allxml[i]['annotation']['object']]
  return bb

bball = [extractbb(i) for i in range(0,len(allxml))]
print(len(bball))

## Vizualisation of pair images, labels

In [ ]:
from deepforest import visualize

In [ ]:
# function to plot images
def cv2_imshow(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    # cv2 stores colors as BGR; convert to RGB
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)

    return plt.imshow(a, **kwargs)

In [ ]:
image = tc_rgb

In [ ]:
# plot predicted bbox
image2 = image.values.copy()
target_bbox = bball[0]

for row in target_bbox:
    cv2.rectangle(image2, (int(row["xmin"]), int(row["ymin"])), (int(row["xmax"]), int(row["ymax"])), (0, 0, 0), thickness=10, lineType=cv2.LINE_AA)

plt.figure(figsize=(15,15))
cv2_imshow(image2)
plt.show()

## Run *DeepForest* pretrained model

In [ ]:
from deepforest import main
from deepforest import IoU

In [ ]:
# load deep forest model
model = main.deepforest()
model.use_release()

In [ ]:
#boxes = model.predict_image(image=image.values)
#print(boxes)